In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

trainDataSet = pd.read_csv('../input/train.csv')
testDataSet = pd.read_csv('../input/test.csv')
trainDataSet.head
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
print (trainDataSet.shape)
print (testDataSet.shape)
print (trainDataSet.columns)

In [ ]:
def cleanText(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = re.sub('\d+', ' ', text)
    text = text.strip(' ')
    return text

In [ ]:
for className in class_names:
    print (trainDataSet[className].value_counts())

In [ ]:
print('Percentage of comments that are not labelled:')
print(len(trainDataSet[(trainDataSet['toxic']==0) & 
             (trainDataSet['severe_toxic']==0) & 
             (trainDataSet['obscene']==0) & 
             (trainDataSet['threat']== 0) & 
             (trainDataSet['insult']==0) &
             (trainDataSet['identity_hate']==0)]) / len(trainDataSet))

In [ ]:
tempToxicDataSet = trainDataSet[trainDataSet['toxic'] == 0][0:1]
tempInsultDataSet = trainDataSet[trainDataSet['toxic'] == 1][0:1]
frames = [tempToxicDataSet, tempInsultDataSet]
tempTrainDataSet = pd.concat(frames)
print (tempTrainDataSet.shape)

tempTestDataSet = testDataSet[0:1]
print (tempTestDataSet.shape)

In [ ]:
train_text = tempTrainDataSet['comment_text']
train_target = tempTrainDataSet.loc[:, class_names]
test_text = tempTestDataSet['comment_text']
all_text = pd.concat([train_text, test_text])
StringData = ""
for i in all_text:
    StringData += i
StringData

|### SENT TOKENIZE - Convert the paragraph into sentences
### WORD TOKENIZE - Convert a sentence or paragraph into words

In [ ]:
print ("Original Text - ",StringData,"\n")
sentences = nltk.sent_tokenize(StringData)
print (len(sentences))
sentences

In [ ]:
words = nltk.word_tokenize(StringData)
print (len(words))
words

#### STEMMING - Stemming is the process of reducing a word to its word stem that affixes to suffixes and prefixes or to the roots of words known as a lemma. Stemming is important in natural language understanding (NLU) and natural language processing (NLP).
#### Example - (Intelligence or Intelligently => Intelligen)
#### Problem - Produced intermediate representation of the word may not have any meaning 
#### LEMMATIZATION - Same as stemming but the intermediate representation has a meaning
#### Example - (Intelligence or Intelligently => Intelligent)

In [ ]:
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
for i in range(len(sentences)):
    print ("Actual Sentence - ",sentences[i],"\n")
    stemmingWords = nltk.word_tokenize(sentences[i])
    lemmatizingWords = nltk.word_tokenize(sentences[i])
    stemmedWords = [stemmer.stem(word) for word in stemmingWords]
    lemmatizedWords = [lemmatizer.lemmatize(word) for word in lemmatizingWords]
    print ("Stemmed Words - \n")
    print (stemmedWords)
    print ("Lemmatized Words - \n")
    print (lemmatizedWords)
    print ("____________________________________________")

### REMOVAL OF STOP WORDS
### Example - (to, be etc....)
### Stopwords are not required while doing sentiment analysis and we need to carry out this step for better performance

In [ ]:
from nltk.corpus import stopwords

for i in range(len(sentences)):
    print ("Actual Sentence - ",sentences[i],"\n")
    words = nltk.word_tokenize(sentences[i])
    newwords = [word for word in words if word not in stopwords.words('english')]
    print (newwords)
    print ("____________________________________________")

### PARTS OF SPEECH TAGGING

In [ ]:
words = nltk.word_tokenize(StringData)
tagged_words = nltk.pos_tag(words)
words_tags = []
for tw in tagged_words:
    words_tags.append(tw[0]+"_"+tw[1])
print (words_tags)

### NAMED ENTITY RECOGNITION

In [ ]:
import matplotlib

words = nltk.word_tokenize(StringData)
tagged_words = nltk.pos_tag(words)
namedEntity = nltk.ne_chunk(tagged_words)
print (namedEntity)


### BAG OF WORDS MODEL
### The bag-of-words model is a way of representing text data when modeling text with machine learning algorithms.
### Study Link  - https://machinelearningmastery.com/gentle-introduction-bag-words-model/


1. Convert all letters to lower case
1. Remove all special characters
1. Remove all extra spaces

In [ ]:
import re

sentences = nltk.sent_tokenize(StringData)
for i in range(len(sentences)):
    sentences[i] = sentences[i].lower()
    sentences[i] = re.sub(r'\W',' ',sentences[i])
    sentences[i] = re.sub(r'\s+',' ',sentences[i])
sentences

### 1. Create the word count dictionary
### 2. Get the first 5 frequently occuring words.

In [ ]:
word2count = {}
for data in sentences:
    words = nltk.word_tokenize(data)
    for word in words:
        if word not in word2count.keys():
            word2count[word] = 1
        else:
            word2count[word] += 1
import heapq
freq_words = heapq.nlargest(30, word2count, key=word2count.get)
freq_words

### Create the bag of words model

In [ ]:
bagOfWords = []
for data in sentences:
    vector = []
    for word in freq_words:
        if word in nltk.word_tokenize(data):
            vector.append(1)
        else:
            vector.append(0)
    bagOfWords.append(vector)
bagOfWords = np.asarray(bagOfWords)
bagOfWords

### Drawbacks of Bag Of Words model -
### 1.  All words have same importance. 
### 2. No semantic information is preserved
### Solution to the above problems is - TF-IDF model (Term Frequency - Inverse Document Frequency )
### TF-IDF = (Number of occurences of word in the document / Total Number of words in a document)


In [ ]:
# IDF Matrix
word_idfs = {}
for word in freq_words:
    document_count = 0
    for data in sentences:
        if word in nltk.word_tokenize(data):
            document_count += 1
    word_idfs[word] = np.log((len(sentences)/document_count)+1) # +1 is the bias and standard way of calulating TF-IDF
word_idfs

In [ ]:
# TF Matrix
tf_matrix = {}
for word in freq_words:
    doc_tf = []
    for data in sentences:
        frequency = 0
        for w in nltk.word_tokenize(data):
            if w == word:
                frequency += 1
        tf_word = frequency/len(nltk.word_tokenize(data))
        doc_tf.append(tf_word)
    tf_matrix[word] = doc_tf
# tf_matrix

In [ ]:
# TF_IDF MATRIX
tfidf_matrix = []
for word in tf_matrix.keys():
    tfidf = []
    for value in tf_matrix[word]:
        score = value * word_idfs[word]
        tfidf.append(score)
    tfidf_matrix.append(tfidf)
tfidf_matrix

In [ ]:
X = np.asarray(tfidf_matrix)
X = np.transpose(X)
X.shape

####  N-GRAM MODEL - Need to know the Markov chains.  An n-gram is a contiguous sequence of n items from a given sample of text or speech. N-grams of texts are extensively used in text mining and natural language processing tasks. They are basically a set of co-occuring words within a given window and when computing the n-grams you typically move one word forward (although you can move X words forward in more advanced scenarios). For example, for the sentence "The cow jumps over the moon". If N=2 (known as bigrams), then the ngrams would be:
* the cow
* cow jumps
* jumps over
* over the
* the moon

#### Generating a word n-gram in the given code

In [ ]:
import random

text = train_text[0]
words = nltk.word_tokenize(text)
print (text)
print ("\n")
ngrams = {}
n = 3
for i in range(len(words) - n):
    gram = ' '.join(words[i:i+n])
    if gram not in ngrams.keys():
        ngrams[gram] = []
    ngrams[gram].append(words[i+n])

currentgram = ' '.join(words[0:n])
result = currentgram
for i in range(10):
    if currentgram not in ngrams.keys():
        break
    possibilities = ngrams[currentgram]
    nextItem = possibilities[random.randrange(len(possibilities))]
    result  += ' '+nextItem
    rwords = nltk.word_tokenize(result)
    currentGram = ' '.join(rwords[len(rwords)-n:len(rwords)])
    
print (ngrams)

## LATENT SYMANTIC ANALYSIS (LSA) - Latent Semantic Analysis is a technique for creating a vector representation of a document. Having a vector representation of a document gives you a way to compare documents for their similarity by calculating the distance between the vectors. This in turn means you can do handy things like classifying documents to determine which of a set of known topics they most likely belong to.
#### We need to build SVD model . SVD - Singular Value Decomposition. Refer to the link for more information - https://machinelearningmastery.com/singular-value-decomposition-for-machine-learning/
#### LSA  is used for following applications:
#### 1.  Article Bucketing in websites
#### 2. Finding relationships between articles/words
#### 3. Page indexing in search engines

In [ ]:
from sklearn.decomposition import TruncatedSVD

dataset = ["The amount of population is increasing day by day",
           "The concert was great",
           "I love to see Gordan Ramsay cook",
           "Google is introducing a new technology",
           "AI Robots are the example of great technology present today",
           "All of us were singing in the concert",
           "We have launch campaigns to stop pollution and global warming"
          ]
dataset = [line.lower() for line in dataset]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(dataset)
#print (X[0]) # (documentNumber , position) tfidf value

lsa = TruncatedSVD(n_components = 4, n_iter = 100) # n_components are the number of concepts that you want to find from the data
lsa.fit(X)
#print (lsa.components_[3])
terms = vectorizer.get_feature_names()
concept_words = {}
for i,comp in enumerate(lsa.components_):
    componentTerms = zip(terms,comp)
    sortedTerms = sorted(componentTerms, key=lambda x:x[1], reverse=True)
    sortedTerms =  sortedTerms[:10]
    concept_words["Concept "+str(i)] = sortedTerms

for key in concept_words.keys():
    sentence_scores = []
    for sentence in dataset:
        words = nltk.word_tokenize(sentence)
        score = 0
        for word in words:
            for word_with_score in concept_words[key]:
                if word == word_with_score[0]:
                    score += word_with_score[1]
        sentence_scores.append(score)
    print("\n"+key+"\n")
    for sent_score in sentence_scores:
        print (sent_score)

#### Find the synonyms and antonyms for words - Refer to link - https://wordnet.princeton.edu/

In [ ]:
from nltk.corpus import wordnet

synonyms = []
antonyms = []

for syn in wordnet.synsets("good"):
    for s in syn.lemmas():
        synonyms.append(s.name())
        for a in s.antonyms():
            antonyms.append(a.name())
print (set(antonyms))
print (set(synonyms))

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

wc = WordCloud().generate(StringData)
plt.figure(figsize=(15,15))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()


In [ ]:
from nltk.corpus import stopwords
import string

oneSetOfStopWords = set(stopwords.words('english')+['``',"''"])
totalWords =[]
Sentences = trainDataSet['comment_text'].values
cleanedSentences = ""
for i in range(0,5000):
    cleanedText = cleanText(Sentences[i])
    cleanedSentences += cleanedText
    requiredWords = nltk.word_tokenize(cleanedText)
    for word in requiredWords:
        if word not in oneSetOfStopWords and word not in string.punctuation:
            totalWords.append(word)
    
wordfreqdist = nltk.FreqDist(totalWords)
mostcommon = wordfreqdist.most_common(50)
print(mostcommon)

wc = WordCloud().generate(cleanedSentences)
plt.figure(figsize=(15,15))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
import string
print (string.punctuation)